In [27]:
# !pip3 install langchain_community

In [28]:
from langchain_community.llms import Ollama

In [29]:
llm = Ollama(model="llama2")

'\nThe first man to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" as he became the first person to set foot on the lunar surface.'

In [91]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the categories for each expense:\n\nSpotify AB by Adyen - Entertainment\nBeta Boulders Ams Amsterdam Nld - Sports\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Groceries\nBistro Bar Amsterdam - Food and Beverage'

### Read transaction data

In [92]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (USD)
0,2/14/2024,HCA Healthcare,Expense,2183.60
1,2/10/2024,7-ELEVEN 39216 00073FRISCO TX,Expense,10.80
2,2/9/2024,CVS/PHARMACY #10581 FRISCO TX,Expense,9.23
3,2/9/2024,GOTHRU MEDIA TORONTO,Expense,23.95
4,2023-12-29,Spotify Ab By Adyen,Expense,12.19


In [93]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

5

In [94]:
unique_transactions[1:10]

array(['7-ELEVEN 39216 00073FRISCO              TX',
       'CVS/PHARMACY #10581 FRISCO              TX',
       'GOTHRU MEDIA        TORONTO', 'Spotify Ab By Adyen'], dtype=object)

### Categorise bank transactions with Llama2

In [95]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 5]

In [96]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [97]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Of course! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Groceries', '6. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Groceries,5. Etos AMSTERDAM NLD,Groceries
8,6. Bistro Bar Amsterdam - Dining,6. Bistro Bar Amsterdam,Dining


In [98]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. HCA Healthcare - Healthcare,3. HCA Healthcare,Healthcare
5,4. 7-ELEVEN 39216 - Retail,4. 7-ELEVEN 39216,Retail
6,5. CVS/PHARMACY #10581 - Healthcare,5. CVS/PHARMACY #10581,Healthcare
7,6. GOTHRU MEDIA - Media,6. GOTHRU MEDIA,Media


In [ ]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Sport', 'Healthcare', 'Retail', 'Media'],
      dtype=object)

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\6041183\AppData\Local\Temp\ipykernel_20140\2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. HCA Healthcare - Healthcare,3. HCA Healthcare,Health and Wellness
5,4. 7-ELEVEN 39216 - Retail,4. 7-ELEVEN 39216,Retail
6,5. CVS/PHARMACY #10581 - Healthcare,5. CVS/PHARMACY #10581,Health and Wellness
7,6. GOTHRU MEDIA - Media,6. GOTHRU MEDIA,Media


In [ ]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (USD),Transaction vs category,Transaction,Category
0,2/14/2024,HCA Healthcare,Expense,2183.60,NaN,NaN,NaN
1,2/10/2024,7-ELEVEN 39216 00073FRISCO TX,Expense,10.80,NaN,NaN,NaN
2,2/9/2024,CVS/PHARMACY #10581 FRISCO TX,Expense,9.23,NaN,NaN,NaN
3,2/9/2024,GOTHRU MEDIA TORONTO,Expense,23.95,NaN,NaN,NaN
4,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN


In [ ]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)